In [3]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
from random import randint
from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA

# from myUtils import drop_symbols_all_NaN, yf_symbols_close, chunked_list # NOQA
from myUtils import drop_symbols_all_NaN, yf_symbols_close_1, chunked_list # NOQA

from myUtils import yf_download_AdjOHLCV, yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output
# verbose = True  # True prints more output

# path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_dir = 'C:/Users/ping/Desktop/trash/'
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'symbols_mktCap_2b.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded'  # pickled filename
filename_pickled_df_OHLCV = 'df_OHLCV'  # pickled filename reindexed to NYSE dates
filename_pickled_df_symbols_close = "df_symbols_close"  # pickled filename
filename_pickled_symbols_df_OHLCV =  'symbols_df_OHLCV'  # pickled filename

In [32]:
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

In [ ]:
def download_n_adjOHLCV (symbols):
  from myUtils import yf_download_AdjOHLCV_noAutoAdj

  df_list=[]
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)
  df_list.append(df)
  df = pd.concat(df_list, axis=1)
  # get unique symbols from column name
  l_symbols = list(set(df.columns.get_level_values(0)))
  l_symbols.sort()

  df_adj_list=[]
  # adjust OHLC using 'Adj Close'
  for symbol in _l_symbols:
    df1 = auto_adjust(_df[symbol])
    df_adj_list.append(_df1)

  df_adj = pd.concat(_df_adj_list, axis=1)
  cols = ['Open', 'High', 'Low', 'Close', 'Volume']
  # create multilevel column names
  col_names = pd.MultiIndex.from_product([l_symbols, cols])
  df_adj.columns = col_names

  return df_adj

In [26]:
test_symbols_chunks = [['A', 'SHEL'], ['YUM', 'AAPL']]  # must be more than 1 symbol in each chuck, otherwise symbol won't appear in header

In [34]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(test_symbols_chunks):

######################## NOT ADJUSTING OHLC CORRECT, CHECK USING # verify df test_symbols' close against Yahoo
  # df = yf_download_AdjOHLCV(symbols, verbose=False)
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  # df = auto_adjust(df)
  # https://github.com/ranaroussi/yfinance/blob/3f33aa0377c9feff68f25345ce5603d52d7359d5/yfinance/utils.py
######################## NOT ADJUSTING OHLC CORRECT, CHECK USING # verify df test_symbols' close against Yahoo  

  # symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  
  df_list.append(df)
  # # pause 5 - 15 sec between download
  # if i < len(symbols_chunks) - 1 :  # skip pause after last download
  #   print(f'downloaded symbols from chuck {i}, sleep start')
  #   # sleep 78(18m 25s), 155, 305 sec to avoid download error from Yahoo
  #   time.sleep(78)
  #   print(f'downloaded symbols from chuck {i}, sleep ends')
  # else:
  #   print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  2 of 2 completed
symbols_download_err: []


In [35]:
df = pd.concat(df_list, axis=1)

In [36]:
df

A                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
1980-12-12         NaN         NaN         NaN         NaN         NaN   
1980-12-15         NaN         NaN         NaN         NaN         NaN   
1980-12-16         NaN         NaN         NaN         NaN         NaN   
1980-12-17         NaN         NaN         NaN         NaN         NaN   
1980-12-18         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2022-10-31  137.690002  138.929993  137.350006  138.350006  138.350006   
2022-11-01  139.500000  141.350006  138.059998  140.889999  140.889999   
2022-11-02  140.610001  140.940002  135.210007  135.270004  135.270004   
2022-11-03  133.770004  135.869995  130.660004  134.460007  134.460007   
2022-11-04  135.649994  137.009995  133.110001  136.080002  136.080002   

                            SHEL                                   ...  \
               Volume       Open       High        Low      Close  ...   
Date                                                               ...   
1980-12-12        NaN        NaN        NaN        NaN        NaN  ...   
1980-12-15        NaN        NaN        NaN        NaN        NaN  ...   
1980-12-16        NaN        NaN        NaN        NaN        NaN  ...   
1980-12-17        NaN        NaN        NaN        NaN        NaN  ...   
1980-12-18        NaN        NaN        NaN        NaN        NaN  ...   
...               ...        ...        ...        ...        ...  ...   
2022-10-31  1307600.0  55.099998  55.994999  54.875000  55.630001  ...   
2022-11-01  1182900.0  56.919998  56.950001  55.799999  55.840000  ...   
2022-11-02  1806100.0  55.950001  56.044998  54.840000  54.910000  ...   
2022-11-03  1199400.0  54.779999  55.869999  54.598999  55.730000  ...   
2022-11-04  1272100.0  57.279999  57.665001  56.660000  57.060001  ...   

                   YUM                                           AAPL  \
                   Low       Close   Adj Close     Volume        Open   
Date                                                                    
1980-12-12         NaN         NaN         NaN        NaN    0.128348   
1980-12-15         NaN         NaN         NaN        NaN    0.122210   
1980-12-16         NaN         NaN         NaN        NaN    0.113281   
1980-12-17         NaN         NaN         NaN        NaN    0.115513   
1980-12-18         NaN         NaN         NaN        NaN    0.118862   
...                ...         ...         ...        ...         ...   
2022-10-31  116.400002  118.250000  118.250000  2193600.0  153.160004   
2022-11-01  117.519997  118.150002  118.150002  2361600.0  155.080002   
2022-11-02  116.720001  117.370003  117.370003  2543800.0  148.949997   
2022-11-03  116.190002  119.500000  119.500000  2546300.0  142.059998   
2022-11-04  120.250000  121.779999  121.779999  2850100.0  142.089996   

                                                                       
                  High         Low       Close   Adj Close     Volume  
Date                                                                   
1980-12-12    0.128906    0.128348    0.128348    0.099874  469033600  
1980-12-15    0.122210    0.121652    0.121652    0.094663  175884800  
1980-12-16    0.113281    0.112723    0.112723    0.087715  105728000  
1980-12-17    0.116071    0.115513    0.115513    0.089886   86441600  
1980-12-18    0.119420    0.118862    0.118862    0.092492   73449600  
...                ...         ...         ...         ...        ...  
2022-10-31  154.240005  151.919998  153.339996  153.086044   97943200  
2022-11-01  155.449997  149.130005  150.649994  150.400497   80379300  
2022-11-02  152.169998  145.000000  145.029999  144.789810   93604600  
2022-11-03  142.800003  138.750000  138.880005  138.650009  

In [31]:
df1 = df.YUM
df1.loc['2021-12-15':'2021-12-25']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-15,132.009995,132.880005,131.160004,132.419998,130.518158,1226000.0
2021-12-16,132.929993,134.000000,132.729996,133.589996,131.671356,1022400.0
2021-12-17,133.410004,133.839996,131.830002,131.929993,130.035187,2491600.0
2021-12-20,130.690002,131.720001,130.119995,131.570007,129.680359,1155400.0
2021-12-21,132.389999,134.490005,132.190002,133.369995,131.454498,1202400.0
2021-12-22,133.619995,135.139999,133.500000,134.630005,132.696426,940100.0
2021-12-23,135.490005,135.830002,134.979996,135.339996,133.396210,958200.0


In [100]:
test_symbols = ['A', 'SHEL', 'YUM', 'AAPL']
for symbol in test_symbols:

  s = df.iloc[-222]
  sDate = s.name.strftime('%Y-%m-%d')
  sClose = s[symbol].Close

  df_sym = yf.Ticker(symbol).history(period='2y')
  yhClose = df_sym.loc[sDate]['Close']
  
  abs_pct_diff = abs(1 - sClose/yhClose)*100
  print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
  Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
  if abs_pct_diff > .0001:
    msg_stop = f'{symbol}  %_dif_Close > .0001'
    raise SystemExit(msg_stop)
  if symbol == test_symbols[-1]:
    msg_done = f"No errors found.  df test_symbols' Close matched Yahoo symbols' Close "
    print(msg_done)

symbol:     A   Date: 2021-12-20   df_Close:  149.46280   Yahoo_Close:  149.46280   %_dif_Close: 0.00000
symbol:  SHEL   Date: 2021-12-20   df_Close:   41.17991   Yahoo_Close:   41.17992   %_dif_Close: 0.00001
symbol:   YUM   Date: 2021-12-20   df_Close:  129.68037   Yahoo_Close:  129.68037   %_dif_Close: 0.00000
symbol:  AAPL   Date: 2021-12-20   df_Close:  168.77049   Yahoo_Close:  168.77049   %_dif_Close: 0.00000
No errors found.  df test_symbols' Close matched Yahoo symbols' Close 


In [70]:
l_symbols = list(set(df.columns.get_level_values(0)))
l_symbols.sort()
df_adj_list=[]

for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

In [73]:
df_adj_list

[                  Open        High         Low       Close     Volume
 Date                                                                 
 1980-12-12         NaN         NaN         NaN         NaN        NaN
 1980-12-15         NaN         NaN         NaN         NaN        NaN
 1980-12-16         NaN         NaN         NaN         NaN        NaN
 1980-12-17         NaN         NaN         NaN         NaN        NaN
 1980-12-18         NaN         NaN         NaN         NaN        NaN
 ...                ...         ...         ...         ...        ...
 2022-10-31  137.690002  138.929993  137.350006  138.350006  1307600.0
 2022-11-01  139.500000  141.350006  138.059998  140.889999  1182900.0
 2022-11-02  140.610001  140.940002  135.210007  135.270004  1806100.0
 2022-11-03  133.770004  135.869995  130.660004  134.460007  1199400.0
 2022-11-04  135.649994  137.009995  133.110001  136.080002  1272100.0
 
 [10565 rows x 5 columns],
                   Open        High         Low 

In [72]:
df_adj = pd.concat(df_adj_list, axis=1)
df_adj

,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume
Date,,,,,,,,,,,,,,,,,,,,
1980-12-12,NaN,NaN,NaN,NaN,NaN,0.099874,0.100308,0.099874,0.099874,469033600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,NaN,NaN,NaN,NaN,NaN,0.095098,0.095098,0.094663,0.094663,175884800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-16,NaN,NaN,NaN,NaN,NaN,0.088149,0.088149,0.087715,0.087715,105728000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-17,NaN,NaN,NaN,NaN,NaN,0.089886,0.090320,0.089886,0.089886,86441600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-18,NaN,NaN,NaN,NaN,NaN,0.092492,0.092926,0.092492,0.092492,73449600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,137.690002,138.929993,137.350006,138.350006,1307600.0,152.906350,153.984563,151.668398,153.086044,97943200,55.099998,55.994999,54.875000,55.630001,5838825.0,117.580002,119.459999,116.400002,118.250000,2193600.0
2022-11-01,139.500000,141.350006,138.059998,140.889999,1182900.0,154.823169,155.192551,148.883026,150.400497,80379300,56.919998,56.950001,55.799999,55.840000,4792770.0,119.010002,119.360001,117.519997,118.150002,2361600.0
2022-11-02,140.610001,140.940002,135.210007,135.270004,1806100.0,148.703316,151.917985,144.759861,144.789810,93604600,55.950001,56.044998,54.840000,54.910000,3919461.0,120.199997,120.279999,116.720001,117.370003,2543800.0


In [92]:
l_symbols
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
idx_mult = pd.MultiIndex.from_product([l_symbols, cols])
idx_mult
_df.columns = idx_mult


In [93]:
_df

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
1980-12-12         NaN         NaN         NaN         NaN        NaN   
1980-12-15         NaN         NaN         NaN         NaN        NaN   
1980-12-16         NaN         NaN         NaN         NaN        NaN   
1980-12-17         NaN         NaN         NaN         NaN        NaN   
1980-12-18         NaN         NaN         NaN         NaN        NaN   
...                ...         ...         ...         ...        ...   
2022-10-31  137.690002  138.929993  137.350006  138.350006  1307600.0   
2022-11-01  139.500000  141.350006  138.059998  140.889999  1182900.0   
2022-11-02  140.610001  140.940002  135.210007  135.270004  1806100.0   
2022-11-03  133.770004  135.869995  130.660004  134.460007  1199400.0   
2022-11-04  135.649994  137.009995  133.110001  136.080002  1272100.0   

                  AAPL                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
1980-12-12    0.099874    0.100308    0.099874    0.099874  469033600   
1980-12-15    0.095098    0.095098    0.094663    0.094663  175884800   
1980-12-16    0.088149    0.088149    0.087715    0.087715  105728000   
1980-12-17    0.089886    0.090320    0.089886    0.089886   86441600   
1980-12-18    0.092492    0.092926    0.092492    0.092492   73449600   
...                ...         ...         ...         ...        ...   
2022-10-31  152.906350  153.984563  151.668398  153.086044   97943200   
2022-11-01  154.823169  155.192551  148.883026  150.400497   80379300   
2022-11-02  148.703316  151.917985  144.759861  144.789810   93604600   
2022-11-03  141.824736  142.563516  138.520220  138.650009   97918500   
2022-11-04  142.089996  142.669998  134.380005  138.380005  140716700   

                 SHEL                                                     YUM  \
                 Open       High        Low      Close     Volume        Open   
Date                                                                            
1980-12-12        NaN        NaN        NaN        NaN        NaN         NaN   
1980-12-15        NaN        NaN        NaN        NaN        NaN         NaN   
1980-12-16        NaN        NaN        NaN        NaN        NaN         NaN   
1980-12-17        NaN        NaN        NaN        NaN        NaN         NaN   
1980-12-18        NaN        NaN        NaN        NaN        NaN         NaN   
...               ...        ...        ...        ...        ...         ...   
2022-10-31  55.099998  55.994999  54.875000  55.630001  5838825.0  117.580002   
2022-11-01  56.919998  56.950001  55.799999  55.840000  4792770.0  119.010002   
2022-11-02  55.950001  56.044998  54.840000  54.910000  3919461.0  120.199997   
2022-11-03  54.779999  55.869999  54.598999  55.730000  3795150.0  116.949997   
2022-11-04  57.279999  57.665001  56.660000  57.060001  6534672.0  121.150002   

                                                           
                  High         Low       Close     Volume  
Date                                                       
1980-12-12         NaN         NaN         NaN        NaN  
1980-12-15         NaN         NaN         NaN        NaN  
1980-12-16         NaN         NaN         NaN        NaN  
1980-12-17         NaN         NaN         NaN        NaN  
1980-12-18         NaN         NaN         NaN        NaN  
...                ...         ...         ...        ...  
2022-10-31  119.459999  116.400002  118.250000  2193600.0  
2022-11-01  119.360001  117.519997  118.150002  2361600.0  
2022-11-02  120.279999  116.720001  117.370003  2543800.0  
2022-11-03  120.639999  116.190002  119.500000  2546300.0  
2022-11-04  122.419998  120.250000  121.779999  2850100.0  


In [85]:
_df = df_adj.copy()
_df

,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume,Open,High,Low,Close,Volume
Date,,,,,,,,,,,,,,,,,,,,
1980-12-12,NaN,NaN,NaN,NaN,NaN,0.099874,0.100308,0.099874,0.099874,469033600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,NaN,NaN,NaN,NaN,NaN,0.095098,0.095098,0.094663,0.094663,175884800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-16,NaN,NaN,NaN,NaN,NaN,0.088149,0.088149,0.087715,0.087715,105728000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-17,NaN,NaN,NaN,NaN,NaN,0.089886,0.090320,0.089886,0.089886,86441600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-18,NaN,NaN,NaN,NaN,NaN,0.092492,0.092926,0.092492,0.092492,73449600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,137.690002,138.929993,137.350006,138.350006,1307600.0,152.906350,153.984563,151.668398,153.086044,97943200,55.099998,55.994999,54.875000,55.630001,5838825.0,117.580002,119.459999,116.400002,118.250000,2193600.0
2022-11-01,139.500000,141.350006,138.059998,140.889999,1182900.0,154.823169,155.192551,148.883026,150.400497,80379300,56.919998,56.950001,55.799999,55.840000,4792770.0,119.010002,119.360001,117.519997,118.150002,2361600.0
2022-11-02,140.610001,140.940002,135.210007,135.270004,1806100.0,148.703316,151.917985,144.759861,144.789810,93604600,55.950001,56.044998,54.840000,54.910000,3919461.0,120.199997,120.279999,116.720001,117.370003,2543800.0


In [9]:
# verify df test_symbols' close against Yahoo
# test_symbols = ['A', 'SHEL', 'YUM']
# test_symbols = ['AAPL']
now = datetime.now()  # get current date and time
# if now.hour >= 20:  # only run this test after 8 pm when Yahoo data should be updated
#  run this test after 8 pm or on the weekend when Yahoo data should be updated
if (now.hour >= 20) or now.weekday() == 5 or now.weekday() == 6:
  for symbol in test_symbols:
    s = df.iloc[-222]
    sDate = s.name.strftime('%Y-%m-%d')

    # sClose = s[symbol].close
    sClose = s[symbol].Close
    sClose
    df_sym = yf.Ticker(symbol).history(period='2y')
    yhClose = df_sym.loc[sDate]['Close']
    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_stop = f'{symbol}  %_dif_Close > .0001'
      raise SystemExit(msg_stop)
    if symbol == test_symbols[-1]:
      msg_done = f"No errors found.  df test_symbols' Close matched Yahoo symbols' Close "
      print(msg_done)
else:
  print(f"Did not verify df test_symbols' close against Yahoo.  It's not 8 PM yet. Yahoo may not have updated their data.")

Did not verify df test_symbols' close against Yahoo.  It's not 8 PM yet. Yahoo may not have updated their data.
